<div style="border:1px solid black; padding:10px 10px;">
    <strong>CIVIL-321 "Modélisation Numérique des Solides et Structures"</strong><br/><br/>
    <span style="text-decoration:underline;font-weight:bold;">Comment utiliser ce Jupyter Notebook?
    </span><br/><br/>
    Ce <strong>Notebook</strong> est constitué de cellules de texte et de cellule de code. Les cellules de codes doivent être  <strong>executées</strong> pour voir le résultat du programme. Certaines cellules doivent être remplies par vos soins. Pour exécuter une cellule, cliquez dessus simplement et ensuite cliquez sur le bouton "play" (<span style="font: bold 12px/30px Arial, serif;">&#9658;</span>) dans la barre de menu au dessus du notebook. Vous pouvez aussi taper la combinaison de touches <code>shift + enter</code>. Il est important d'éxécuter les cellules de code en respectant leur ordre d'arrivée dans le notebook.
</div>

On vous encourage à poser vos questions et donner votre feedback sur ce notebook sur la plateforme ED Discussion du cours accessible en cliquant sur ce bouton:
 
 
 
<div class="container" >
        <a href="https://edstem.org/eu/courses/409/discussion?category=Exercices">
            <button class="btn btn-primary btn-lg">Ed Discussion</button>
        </a>
</div>

In [ ]:
import numpy as np
from plot import *


# Barre modélisée comme un milieu continu

Dans cette série, nous allons résoudre un problème de milieu continu (une poutre console) et manipuler des maillages automatiquement générés par Gmsh, un mailleur open-source.

## Poutre bi-dimensionnelle
*Géométrie et conditions limites.*
La poutre console montrée sur la Figure a une longeur $L_{x} = 500$ mm, une hauteur $L_{y} = 100$ mm et épaisseur $t = 10$ mm.
On encastre complètement la partie gauche de la poutre, et on charge le coin supérieur droit avec une force $P = 100$ N. Les constantes d'élasticité sont $E = 210$ GPa et $\nu = 0.3$.
![](Images/fig1.svg)
La structure est maillée avec des éléments triangulaires linéaires (déformation constante). Le maillage est structuré selon deux directions, comme le montrent les figure ci dessous, avec $n_{x}=15$ et $n_{y}=3$ les nombres d'éléments selon $x$ et $y$ respectivement:

![](Images/fig2.svg)
*Numérotation des noeuds pour $n_{x} = 15$ et $n_{y}$ = 3.*


1. Déterminer les espacements $l_{x}$ et $l_y$ entre deux noeuds consécutifs, ainsi que les positions des noeuds 1, 50 et 63 montrés sur la figure ci-dessus, numérotés comme indiqué.

---

 **Place your answer here** 

 ---




2. Déterminer les coordonnées du noeud $n$ pour $n_x$ et $n_y$ arbitraires.


---

 **Place your answer here** 

 ---



2. Écrire le code Python calculant la matrice des coordonnées pour des $n_x$ et $n_y$ arbitraires (cette matrice a deux colonnes, pour les positions $x$ et $y$). 

*indice: il est intéressant de trouver la relation entre les coordonnées dans la grille $i, j$ d'un noeud et son index global $n$ . Par exemple le noeud $18$ a pour coordonnées $(2, 1)$* 

In [ ]:
# Place your answer here

## connectivité

![](Images/fig2.svg)

![](Images/fig3.svg)
*Numérotation des éléments pour $n_{x} = 15$ et $n_{y}$ = 3.*

3. Déterminer la connectivité des éléments 0, 30 et 45 pour la numérotation indiquée.

---

 **Place your answer here** 

 ---



3. Déterminer la connectivité d'un élément $e$ pour $n_x$ et $n_y$ arbitraires.

---

 **Place your answer here** 

 ---



4. Implémentez le code Python construisant la matrice de connectivité. Controlez votre mesh avec la routine `plotMesh(coords, conn)`

In [ ]:
# Place your answer here

5. Implémentez le code Python construisant la matrice des numéros d'équation.

In [ ]:
# Place your answer here

## Rigidité

5. Implémenter une fonction calculant la matrice de rigidité locale $K^e$ d'un élément $e$.

In [ ]:

def calculerD(E, nu, contraintes_planes=True):
    # remplir ici
    pass


def calculerB(connectivite_element, coordonnees):
    # remplir ici
    B = None # B matrix
    A = None # area of current triangular element
    return B, A


def calculerMatriceRigiditeLocale(E, nu, t, connectivite_element, coordonnees):
    D = calculerD(E, nu)

    # On calcule B et l'aire du triangle pour l'intégrale
    [B, A] = calculerB(connectivite_element, coordonnees)

    #remplir ici
    pass
    #return K_locale

E = 210e3
t = 10.
nu = 0.3

# plotting local matrix of element 0
Kloc = calculerMatriceRigiditeLocale(E, nu, t, conn[0], coords)
Kloc

In [ ]:
# Place your answer here

6.Adapter la fonction `assemblerMatriceRigidite()` pour assembler la matrice de rigidité globale du système *K*.

In [ ]:
# Cette routine d'assemblage est celle de l'exercice 02_BarresTreillis/02_Exercice.ipynb
# il faut la modifier

def AssemblerMatriceRigidite(k_vec, positions, conn, nb_elem, nb_ddl_tot, eqn_elem):
    K = np.zeros((nb_ddl_tot, nb_ddl_tot))
    
    for e in range(nb_elem):
        # Construction de la matrice de rigidité locale
        Kl = calculerMatriceRigiditeLocale(k_vec[e])
        # Construction de la matrice de rotation
        R = CalculerMatriceRotation(e, positions, conn)
        # Rotation de la matrice dans le système global de coordonnée
        Kg = R@Kl@R.T
        
        # Assemblage dans la matrice de rigidité du système à l'aide des
        idx = eqn_elem[e, :]
        for i, gi in enumerate(idx):
            for j, gj in enumerate(idx):
                K[gi, gj] += Kg[i, j]
    return K

# un routine de cette forme est espérée:

def assemblerMatriceRigidite(E, nu, t, connectivite, coordonnees):
    # remplir ici
    pass

In [ ]:
# Place your answer here

6. Déterminer, pour $n_x$ et $n_y$ arbitraires, quels sont les degrés de liberté bloqués par la condition limite d'encastrement ? Implémenter une fonction `calculerBlocages()` qui retourne un vecteur de taille $n_{nodes} \times 2$ qui contient `True` si le degré de liberté est bloqué et `False` sinon.

In [ ]:
# Place your answer here

7. Calculer la valeur des déplacements nodaux en utilisant $n_x = 15$ et $n_y = 3$ et $P = 100$. Vous    pouvez visualiser votre résultat en utilisant la routine `plotMesh(coords, conn, u)`

In [ ]:
nx = 15
ny = 3
n_nodes = (nx+1)*(ny+1)

# Vecteur déplacements
u = np.zeros(2*(nx+1)*(ny+1))

# Vecteur force
P = -100;
F = np.zeros(n_nodes*2)
F[-1] = P

# Résolution du système Ku=F avec conditions limites
u[libres] = np.linalg.solve(K[libres, :][:, libres], F[libres])

plotMesh(coords+u.reshape(n_nodes, 2)*1000, conn, u)

8. Calculer la valeur du déplacement vertical maximum. 

In [ ]:
# Place your answer here

9.Calculer la valeur théorique en utilisant la théorie des poutres de Bernoulli et comparer avec la valeur trouvée par éléments finis.

---

 **Place your answer here** 

 ---



8. Faire varier le nombre d'éléments dans les directions $x$ et $y$ pour obtenir une valeur du déplacement qui diffère de moins de 1\% de la valeur obtenue par la théorie des poutres. Essayer d'optimiser le nombre d'éléments.

In [ ]:
# Place your answer here

9. Déterminer la contrainte maximum le long de l'axe $y$ pour $x = 0$, en utilisant $n_x = 5$ et $n_y = 1$, $n_x = 15$ et $n_y = 3$, puis en utilisant les valeur de $n_x$ et $n_y$ utilisée précédemment. Comparer les valeurs avec la théorie des poutres. Commenter la convergence.

In [ ]:
# Place your answer here

# Format MSH

Dans cet exercice, nous allons utiliser une fonction pour pouvoir charger un maillage déjà existant sur Python. 

1. Le fichier *carre.msh* contient un maillage. Ouvrez *carre.msh* (dans noto ou en téléchargant le fichier). Construisez à la main la connectivité du maillage. Vous pouvez vous aider de la documentation de Gmsh (http://gmsh.info/doc/texinfo/gmsh.html#MSH-file-format-version-2-_0028Legacy_0029).
2. Chargez le maillage avec la fonction `readMesh(carre.msh)`. Vérifier que les coordonnées et connectivité sont correctes. Affichez le maillage avec la fonction `plotMesh`.

In [ ]:
# Place your answer here

3. Téléchargez et installez le mailleur open-source Gmsh (http://gmsh.info/).
4. Ouvrez *carre.msh* avec Gmsh. Pour afficher la numérotation des noeuds et éléments, allez dans *Tools $\Rightarrow$ Options $\Rightarrow$ Mesh $\Rightarrow$ Visibility* et cochez *Nodes*, *Node labels* et *Surface labels*.
5. Supposons que les noeuds de la face supérieure du carré se déplacent horizontalement de $0.1$ m (Sans réaliser de calcul par éléments finis). Affichez la déformée.

In [ ]:
# Place your answer here

# Maillage avec GMSH

  Nous allons utiliser Gmsh afin de mailler la structure suivante à l'aide de triangle T6:
  
  
![](Images/A.svg)

1. A l'aide de Gmsh, nous allons d'abord préciser les contours de la géométrie étudiée (en unité [m]) avec la marche à suivre suivante:
  - Créez un paramètre *lc*, qui sera le paramètre de rafinement du maillage, à l'aide de l'option *Geometry $\Rightarrow$ Elementary entities $\Rightarrow$ Add $\Rightarrow$ Parameter*
  - Créez les points d'angle à l'aide de l'option *Geometry $\Rightarrow$ Elementary entities $\Rightarrow$ Add $\Rightarrow$ Point*. 
    Entrez les coordonnées, et mettez *lc* dans le champ *Prescribed mesh element size at point*.
  - Connectez ces points à l'aide de lignes droites en utilisant l'option *Geometry $\Rightarrow$ Elementary entities $\Rightarrow$ Add $\Rightarrow$ Straight line*.
  - A présent, utilisez ces lignes pour définir la surface de la structure (*Geometry $\Rightarrow$ Elementary entities $\Rightarrow$ Add 
    $\Rightarrow$ Plane surface*). Séléctionnez d'abord le bord extérieur, puis le bord intérieur.
  - Finalement, il faut définir la surface créée comme ``physical entity'' afin qu'elle soit maillée par Gmsh. Pour ce faire, sélectionner la surface créée à l'aide de *Geometry $\Rightarrow$ Physical groups $\Rightarrow$ Add $\Rightarrow$ Surface*.
  - Sauvegardez la géométrie créée avec le nom *solide.geo*. 

2. Ouvrez le fichier *solide.geo* à l'aide d'un éditeur de texte (par exemple le Notepad). Observez le contenu du fichier généré par Gmsh. Vous pouvez modifier le fichier et observer les changements dans Gmsh une fois le fichier ré-ouvert.
3. Afin de mailler la structure, utilisez le menu *Mesh $\Rightarrow$ 2D*. Afin d'utiliser des éléments T6, utilisez l'option *Set order 2*. Sauvegardez le maillage ainsi créé à l'aide de l'option *File $\Rightarrow$ Save Mesh* en utilisant le nom par défaut *solide.msh*. Vous pouvez modifier directement dans Gmsh la valeur du paramètre *lc* pour raffiner le maillage.
4. Vous pouvez répéter les questions Python de l'exercice précédent avec le nouveau maillage *solide.msh*..


In [ ]:
# Place your answer here